In [1]:
import pandas as pd
from multiprocessing import Process
import time
import numpy as np

wow$


In [137]:
data_df = pd.read_pickle('data_df_2015-2020.pickle')
data_df.iloc[:40]


opponent-points-per-game  points-per-game
2014-11-01 Atlanta                          109.0            102.0
           Boston                           105.0            121.0
           Brooklyn                         121.0            105.0
           Charlotte                        106.0            108.0
           Chicago                           97.0            106.0
           Cleveland                        101.5            102.0
           Dallas                           101.5            110.0
           Denver                            79.0             89.0
           Detroit                           93.0             85.0
           Golden State                      77.0             95.0
           Houston                           91.5            106.0
           Indiana                           94.0             96.0
           LA Clippers                      100.5            105.5
           LA Lakers                        115.0            100.0
           Memphis                           95.0            101.0
           Miami                             95.0            107.0
           Milwaukee                         94.5             99.5
           Minnesota                         98.0             99.0
           New Orleans                       84.0            101.0
           New York                          97.0             87.5
           Okla City                         99.5             89.5
           Orlando                          103.0             91.0
           Philadelphia                      98.0             86.0
           Phoenix                           94.0            106.5
           Portland                          96.0            100.0
           Sacramento                        94.5             90.0
           San Antonio                       97.0             95.0
           Toronto                          102.0            109.0
           Utah                             112.0             97.5
           Washington                       102.5            100.0
2014-11-02 Atlanta                          100.5            102.0
           Boston                           104.5            105.5
           Brooklyn                         105.5            103.5
           Charlotte                         88.5             88.5
           Chicago                           99.7            106.0
           Cleveland                        101.5            102.0
           Dallas                           102.3            109.7
           Denver                            90.5             90.0
           Detroit                           96.0             86.7
           Golden State                      90.5            111.0

In [138]:
games_df = pd.read_pickle('games_df_2015-2020.pickle')
games_df

,Date,Home,Away,DidHomeWin
0,2016-11-01,Cleveland,Houston,True
1,2016-11-01,Indiana,LA Lakers,True
2,2016-11-01,Philadelphia,Orlando,False
3,2016-11-01,Detroit,New York,True
4,2016-11-01,Miami,Sacramento,True
...,...,...,...,...
6248,2016-03-31,Houston,Chicago,False
6249,2016-03-31,Indiana,Orlando,False
6250,2016-03-31,New Orleans,Denver,True
6251,2016-03-31,Okla City,LA Clippers,True


In [139]:
training_data = []
for i in range(len(games_df)):
    try:
        game = games_df.loc[i]

        homeTeam, awayTeam = str(game.Home), str(game.Away)
        date_str = str(game.Date.date())
        home_index = (date_str, homeTeam) # Index to get the stats for a team on a certain date from data_df
        away_index = (date_str, awayTeam)
        home_stats = data_df.loc[home_index].to_numpy()
        away_stats = data_df.loc[away_index].to_numpy()
        # array_to_append = np.array([ np.array([home_stats.to_numpy(), away_stats.to_numpy()]), game.DidHomeWin])
        # stats = np.array([home_stats.to_numpy(), away_stats.to_numpy()]) # make a 2d array for model eventually
        training_data.append([home_stats, away_stats, game.DidHomeWin]) # [1, 0 ] if AWAY WON,and [0,1] if HOME WON
    except KeyError as e:
        print(date_str) 
        print(home_index)

2019-11-01
('2019-11-01', 'Brooklyn')
2019-11-01
('2019-11-01', 'Indiana')
2019-11-01
('2019-11-01', 'Orlando')
2019-11-01
('2019-11-01', 'Boston')
2019-11-01
('2019-11-01', 'Chicago')
2019-11-01
('2019-11-01', 'Dallas')
2019-11-01
('2019-11-01', 'Sacramento')
2019-11-01
('2019-11-01', 'Golden State')
2019-11-02
('2019-11-02', 'Okla City')
2019-11-02
('2019-11-02', 'Detroit')
2019-11-02
('2019-11-02', 'Orlando')
2019-11-02
('2019-11-02', 'Memphis')
2019-11-02
('2019-11-02', 'Milwaukee')
2019-11-02
('2019-11-02', 'Washington')
2019-11-02
('2019-11-02', 'Golden State')
2019-11-02
('2019-11-02', 'Portland')
2019-11-03
('2019-11-03', 'Indiana')
2019-11-03
('2019-11-03', 'Miami')
2019-11-03
('2019-11-03', 'New York')
2019-11-03
('2019-11-03', 'San Antonio')
2019-11-03
('2019-11-03', 'Cleveland')
2019-11-03
('2019-11-03', 'LA Clippers')
2019-11-04
('2019-11-04', 'Washington')
2019-11-04
('2019-11-04', 'Brooklyn')
2019-11-04
('2019-11-04', 'Memphis')
2019-11-04
('2019-11-04', 'Minnesota')
201

In [153]:
training_data[0]


[array([ 92.7, 105.3]), array([103.3, 104.3]), True]

In [146]:
def determine_pythag_expectation(points_against, points_for):
    win = (points_for**(13.91)) / (points_for**13.91 + points_against**13.91)
    return win
def get_point_differential(points_against, points_for):
    return points_for-points_against

In [154]:
correct = 0
extremly_wrong = 0
for data in training_data:
    home_expectation = determine_pythag_expectation(data[0][0], data[0][1])
    away_expectation = determine_pythag_expectation(data[1][0], data[1][1])
    #print(f"Home: {home_expectation} Away :{away_expectation} HomeWin: {data[2]}")
    if (data[0][1] >= data[1][1]) and data[2]:
        correct += 1
    else:
        extremly_wrong += 1

print(correct)
print(extremly_wrong)
print(correct/(correct+extremly_wrong))

1800
3542
0.33695245226506926


In [ ]:
home_expectation

In [ ]:
# building the conv net
training_data[0]

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.linear(4, 64)
        self.fc2 = nn.linear(64, 64)
        self.fc3 = nn.linear(64, 64)
        self.fc4 = nn.linear(64, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x

In [ ]:
net = Net()

In [ ]:
X = torch.randn((2,2))
X.view(-1, 4)

In [ ]:
X = torch.Tensor[i[0]]